In [3]:
import pandas as pd
import sqlite3



Reading all the CSV Files

In [5]:
accidents = pd.read_csv("accident.csv")
vehicle = pd.read_csv("vehicle.csv")
person = pd.read_csv("person.csv")
accident_location = pd.read_csv("accident_location.csv")
node = pd.read_csv("node.csv")
accidents.head()

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,ACCIDENT_TYPE,ACCIDENT_TYPE_DESC,DAY_OF_WEEK,DAY_WEEK_DESC,DCA_CODE,DCA_DESC,LIGHT_CONDITION,...,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_NOT_INJ,NO_PERSONS,POLICE_ATTEND,ROAD_GEOMETRY,ROAD_GEOMETRY_DESC,SEVERITY,SPEED_ZONE,RMA
0,T20120004516,2012-02-24,15:30:00,2,Struck Pedestrian,6,Friday,101,PED EMERGES FROM IN FRONT OF PARKED OR STATION...,1,...,0,1,1,2,1,5,Not at intersection,3,50,Local Road
1,T20120007847,2012-04-03,01:57:00,1,Collision with vehicle,3,Tuesday,130,REAR END(VEHICLES IN SAME LANE),5,...,0,1,2,3,1,5,Not at intersection,3,50,Local Road
2,T20120008205,2012-04-07,01:15:00,2,Struck Pedestrian,7,Saturday,105,PED WALKING AGAINST TRAFFIC.,3,...,1,0,1,2,1,5,Not at intersection,2,80,Arterial Highway
3,T20120008420,2012-02-16,09:15:00,8,No collision and no object struck,5,Thursday,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT),1,...,0,1,0,1,2,5,Not at intersection,3,60,Freeway
4,T20120008442,2012-04-10,10:00:00,1,Collision with vehicle,3,Tuesday,121,RIGHT THROUGH,1,...,1,2,0,3,1,2,T intersection,2,70,Arterial Other


In [13]:
accidents.shape

(191779, 23)

Converting all column headers to Lower

In [9]:
tables = [accidents,vehicle, person, accident_location, node]

In [11]:
for i in tables:
    i.columns = i.columns.str.lower()

In [13]:
node.columns

Index(['accident_no', 'node_id', 'node_type', 'amg_x', 'amg_y', 'lga_name',
       'lga name all', 'deg_urban_name', 'latitude', 'longitude',
       'postcode_crash'],
      dtype='object')

In [61]:
def insert_sql(df_name, sql_table):
    """this function inserts the desired table into the db file"""
    conn = sqlite3.connect("vic_crash_data.db")
    df_name.to_sql(
        sql_table,
        conn,
        if_exists="append", # do not overwrite DDL
        index=False
    )
    conn.close()

In [63]:
#dimension table for type of accidents
accident_type = accidents[['accident_type', 'accident_type_desc']].drop_duplicates().reset_index(drop=True)
accident_type.head()

,accident_type,accident_type_desc
0,2,Struck Pedestrian
1,1,Collision with vehicle
2,8,No collision and no object struck
3,4,Collision with a fixed object
4,7,Fall from or in moving vehicle


In [65]:
#dimenstion table for DCA code
dca_code = accidents[['dca_code', 'dca_desc']].drop_duplicates().reset_index(drop=True)
dca_code.head()

,dca_code,dca_desc
0,101,PED EMERGES FROM IN FRONT OF PARKED OR STATION...
1,130,REAR END(VEHICLES IN SAME LANE)
2,105,PED WALKING AGAINST TRAFFIC.
3,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT)
4,121,RIGHT THROUGH


In [67]:
#dimension table for type of road geometry at accident
road_geometry = accidents[['road_geometry', 'road_geometry_desc']].drop_duplicates().reset_index(drop=True)
road_geometry.head()

,road_geometry,road_geometry_desc
0,5,Not at intersection
1,2,T intersection
2,1,Cross intersection
3,4,Multiple intersection
4,3,Y intersection


In [23]:
vehicle.head()

,accident_no,vehicle_id,vehicle_year_manuf,vehicle_dca_code,initial_direction,road_surface_type,road_surface_type_desc,reg_state,vehicle_body_style,vehicle_make,...,trailer_type,vehicle_colour_1,vehicle_colour_2,caught_fire,initial_impact,lamps,level_of_damage,towed_away_flag,traffic_control,traffic_control_desc
0,T20120001479,A,NaN,1.0,N,9.0,Not known,NaN,NaN,NaN,...,H,ZZ,ZZ,0.0,9,2.0,1,2.0,1.0,Stop-go lights
1,T20120002094,C,1997.0,8.0,W,1.0,Paved,V,SEDAN,MITSUB,...,H,RED,ZZ,2.0,5,2.0,1,2.0,0.0,No control
2,T20120002390,A,2001.0,1.0,NE,3.0,Gravel,V,SEDAN,HOLDEN,...,H,GRY,ZZ,2.0,F,9.0,1,2.0,0.0,No control
3,T20120004794,A,NaN,1.0,E,1.0,Paved,NaN,NaN,UNKN,...,H,ZZ,ZZ,2.0,5,9.0,9,2.0,0.0,No control
4,T20120010082,B,2005.0,2.0,S,1.0,Paved,V,SEDAN,TOYOTA,...,H,SIL,ZZ,2.0,R,2.0,4,1.0,1.0,Stop-go lights


In [69]:
#dimension table indicating the type of road surface at the the time of accident
road_surface = vehicle[['road_surface_type', 'road_surface_type_desc']].drop_duplicates().reset_index(drop=True)
road_surface.head()

,road_surface_type,road_surface_type_desc
0,9.0,Not known
1,1.0,Paved
2,3.0,Gravel
3,2.0,Unpaved
4,NaN,NaN


In [71]:
# dimension table representing type of vehicle involved in the accident
vehicle_type = vehicle[['vehicle_type', 'vehicle_type_desc']].drop_duplicates().reset_index(drop=True)
vehicle_type.head()

,vehicle_type,vehicle_type_desc
0,13,Bicycle
1,1,Car
2,99,Not Known
3,3,Taxi
4,2,Station Wagon


In [73]:
# dimension table indicating what traffic control procedures were performed
traffic_control = vehicle[['traffic_control', 'traffic_control_desc']].drop_duplicates().reset_index(drop=True)
traffic_control.head()

,traffic_control,traffic_control_desc
0,1.0,Stop-go lights
1,0.0,No control
2,11.0,Giveway sign
3,15.0,Other
4,9.0,Roundabout


In [33]:
person.head()

,accident_no,person_id,vehicle_id,sex,age_group,inj_level,inj_level_desc,seating_position,helmet_belt_worn,road_user_type,road_user_type_desc,licence_state,taken_hospital,ejected_code
0,T20120001075,A,A,M,26-29,2,Serious injury,D,1.0,2,Drivers,V,Y,0.0
1,T20120001621,C,C,U,Unknown,4,Not injured,D,9.0,2,Drivers,Z,NaN,0.0
2,T20120002142,A,A,F,30-39,4,Not injured,D,9.0,2,Drivers,V,NaN,0.0
3,T20120002846,A,A,F,70+,2,Serious injury,D,1.0,2,Drivers,V,Y,0.0
4,T20120003531,B,B,M,26-29,4,Not injured,D,1.0,2,Drivers,V,NaN,0.0


In [75]:
# dimension table for injury specific details
injury = person[['inj_level', 'inj_level_desc']].drop_duplicates().reset_index(drop=True)
injury.head()

,inj_level,inj_level_desc
0,2,Serious injury
1,4,Not injured
2,3,Other injury
3,1,Fatality


In [77]:
# type of road user that was involved
road_user = person[['road_user_type', 'road_user_type_desc']].drop_duplicates().reset_index(drop=True)
road_user.head()

,road_user_type,road_user_type_desc
0,2,Drivers
1,6,Bicyclists
2,3,Passengers
3,4,Motorcyclists
4,9,Not Known


In [39]:
accident_location.head()

,accident_no,node_id,road_route_1,road_name,road_type,road_name_int,road_type_int,distance_location,direction_location
0,T20230006450,205541,2510.0,QUEENS,ROAD,ARTHUR,STREET,51.0,N
1,T20190018462,589104,5790.0,ELTHAM-YARRA GLEN,ROAD,KANGAROO GROUND-WARRANDYTE,ROAD,7.0,S
2,T20140009989,356149,2820.0,WEST GATE,FREEWAY,BAY,TRAIL,180.0,NW
3,T20220007169,773905,9999.0,BOX,STREET,PAPERBARK,STREET,45.0,W
4,T20170023163,326150,9999.0,WOLLASTON,ROAD,SOMMERVILLE,BOULEVARD,74.0,N


In [41]:
node.head()

,accident_no,node_id,node_type,amg_x,amg_y,lga_name,lga name all,deg_urban_name,latitude,longitude,postcode_crash
0,T20140004812,272797,N,2519784.896,2385924.133,DANDENONG,DANDENONG,MELB_URBAN,-38.027718,145.22534,3175
1,T20140013245,272790,N,2650453.879,2406340.061,WELLINGTON,WELLINGTON,RURAL_VICTORIA,-37.831820,146.70924,3858
2,T20140012386,272808,N,2512490.880,2414413.116,MANNINGHAM,MANNINGHAM,MELB_URBAN,-37.771168,145.14178,3106
3,T20140013456,272819,N,2524439.623,2402043.625,KNOX,KNOX,MELB_URBAN,-37.882380,145.27782,3156
4,T20140013249,272817,N,2518542.169,2390483.203,DANDENONG,DANDENONG,MELB_URBAN,-37.986670,145.21107,3175


In [93]:
# represents the spacial and location specific insights of the node that was involved in the accident
node = (
    node
    .drop(columns=["accident_no", "postcode_crash", "lga name all"])
    .drop_duplicates(subset=["node_id"])
    .reset_index(drop=True)
)
node.head()

,node_id,node_type,amg_x,amg_y,lga_name,deg_urban_name,latitude,longitude
0,272797,N,2519784.896,2385924.133,DANDENONG,MELB_URBAN,-38.027718,145.22534
1,272790,N,2650453.879,2406340.061,WELLINGTON,RURAL_VICTORIA,-37.831820,146.70924
2,272808,N,2512490.880,2414413.116,MANNINGHAM,MELB_URBAN,-37.771168,145.14178
3,272819,N,2524439.623,2402043.625,KNOX,MELB_URBAN,-37.882380,145.27782
4,272817,N,2518542.169,2390483.203,DANDENONG,MELB_URBAN,-37.986670,145.21107


In [45]:
accident_location.head()

,accident_no,node_id,road_route_1,road_name,road_type,road_name_int,road_type_int,distance_location,direction_location
0,T20230006450,205541,2510.0,QUEENS,ROAD,ARTHUR,STREET,51.0,N
1,T20190018462,589104,5790.0,ELTHAM-YARRA GLEN,ROAD,KANGAROO GROUND-WARRANDYTE,ROAD,7.0,S
2,T20140009989,356149,2820.0,WEST GATE,FREEWAY,BAY,TRAIL,180.0,NW
3,T20220007169,773905,9999.0,BOX,STREET,PAPERBARK,STREET,45.0,W
4,T20170023163,326150,9999.0,WOLLASTON,ROAD,SOMMERVILLE,BOULEVARD,74.0,N


In [103]:
accidents.columns

Index(['accident_no', 'accident_date', 'accident_time', 'accident_type',
       'accident_type_desc', 'day_of_week', 'day_week_desc', 'dca_code',
       'dca_desc', 'light_condition', 'node_id', 'no_of_vehicles',
       'no_persons_killed', 'no_persons_inj_2', 'no_persons_inj_3',
       'no_persons_not_inj', 'no_persons', 'police_attend', 'road_geometry',
       'road_geometry_desc', 'severity', 'speed_zone', 'rma'],
      dtype='object')

In [95]:
# master table containing accident specific variables. In this code relevant variables are selected, renamed, variable types modified and PK constraints handled
accidents_df = accidents[[
    'accident_no',
    'accident_date',
    'accident_time',
    'accident_type',
    'day_of_week',
    'dca_code',
    'node_id',
    'no_of_vehicles',
    'no_persons_killed',
    'no_persons_inj_2',
    'no_persons_inj_3',
    'no_persons_not_inj',
    'police_attend',
    'road_geometry',
    'light_condition',
    'severity',
    'speed_zone',
    'rma'
]].copy()

#renaming
accidents_df.rename(columns={
    'no_persons_killed': 'no_killed',
    'no_persons_inj_2': 'no_person_inj2',
    'no_persons_inj_3': 'no_person_inj3',
    'no_persons_not_inj': 'no_person_not_inj',
    'police_attend': 'police_attended'
}, inplace=True)

#converting to date and time
accidents_df['accident_date'] = pd.to_datetime(
    accidents_df['accident_date'], errors='coerce'
).dt.date

accidents_df['accident_time'] = pd.to_datetime(
    accidents_df['accident_time'], errors='coerce'
).dt.time

# converting to numeric types
int_cols = [
    'node_id',
    'no_of_vehicles',
    'no_killed',
    'no_person_inj2',
    'no_person_inj3',
    'no_person_not_inj',
    'speed_zone'
]

for col in int_cols:
    accidents_df[col] = pd.to_numeric(accidents_df[col], errors='coerce')

#Drop rows with missing PK
accidents_df = accidents_df.dropna(subset=['accident_no'])

accidents_df.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_21752\3920561539.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  accidents_df['accident_time'] = pd.to_datetime(


(191779, 18)

In [97]:
# this table contains vehicle level details involved in the accident. 
vehicle_df = vehicle[[
    'accident_no',
    'vehicle_id',
    'vehicle_dca_code',
    'road_surface_type',
    'reg_state',
    'vehicle_type',
    'fuel_type',
    'no_of_cylinders',
    'seating_capacity',
    'total_no_occupants',
    'driver_intent',
    'level_of_damage'
]].copy()

#renaming
vehicle_df.rename(columns={
    'total_no_occupants': 'occupants'
}, inplace=True)

#converting to numeric
int_cols = [
    'no_of_cylinders',
    'seating_capacity',
    'occupants'
]

for col in int_cols:
    vehicle_df[col] = pd.to_numeric(vehicle_df[col], errors='coerce')

# Drop rows violating PK 
vehicle_df = vehicle_df.dropna(subset=['accident_no', 'vehicle_id'])


vehicle_df.shape

(349610, 12)

In [99]:
#contains person specific details of individual involved in the accident
person_df = person[[
    'accident_no',
    'person_id',
    'vehicle_id',
    'sex',
    'age_group',
    'inj_level',
    'seating_position',
    'road_user_type',
    'taken_hospital'
]].copy()

person_df.rename(columns={
    'road_user_type': 'road_user'
}, inplace=True)

#Dropping rows missing composite primary key values
person_df = person_df.dropna(subset=['accident_no', 'person_id'])

#Dropping duplicates based on composite primary key
person_df = person_df.drop_duplicates(subset=['accident_no', 'person_id']).reset_index(drop=True)

person_df.shape

(447599, 9)

Inserting tables in to the DB files, with dimension tables going first

In [55]:
insert_sql(accident_type, "accident_type_desc")
insert_sql(dca_code, "dca_code_desc")
insert_sql(road_geometry, "road_geometry")
insert_sql(road_surface, "road_surface")
insert_sql(vehicle_type, "vehicle_type")
insert_sql(traffic_control, "traffic_control")
insert_sql(injury, "injury")
insert_sql(road_user, "road_user")

In [57]:
insert_sql(node, "road_node")
insert_sql(accident_location, "accident_node")
insert_sql(accidents_df, "accidents")
insert_sql(vehicle_df, "vehicle_info")
insert_sql(person_df, "person_info")